# 02. Preprocessing (전처리)

QC (품질 검사) + 역채점 + Big Five/Ideology/Honesty-Humility 점수 계산을 통합 수행합니다.

## 학습 목표
- 이상 응답 패턴 탐지 및 제외
- 채점 키(superKey696.csv) 기반 점수 계산
- 역채점 처리 이해
- Ideology, Honesty-Humility 복합 척도 계산

## 참조 파일
- `reports/preprocessing_guide.md` - 점수 계산 공식
- `reports/pipeline_context.json` - 이전 단계 결과

In [1]:
# 필요한 라이브러리 설치
%pip install pandas numpy matplotlib -q

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# 라이브러리 임포트
import pandas as pd
import numpy as np
import json
import os
from datetime import datetime
from glob import glob

# 작업 디렉토리 설정
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')
print(f'작업 폴더: {os.getcwd()}')

# reports/ 폴더의 Context 파일들 확인
context_files = glob('reports/*.json')
print(f'참조할 Context 파일들: {context_files}')

# pipeline_context.json 로드
with open('reports/pipeline_context.json', 'r', encoding='utf-8') as f:
    ctx = json.load(f)
print(f"현재 단계: {ctx.get('current_step')}")
print(f"완료된 단계: {ctx.get('steps_completed')}")

작업 폴더: c:\Users\yujin\code\agent_seminar\sapa-demo
참조할 Context 파일들: ['reports\\pipeline_context.json']
현재 단계: preprocess
완료된 단계: ['scan']


---
## Part 1: 데이터 로드 및 Context 기반 설정

In [3]:
# 데이터 로드
data = pd.read_csv('data/raw/sapa_data.csv')
keys = pd.read_csv('data/raw/superKey696.csv', index_col=0)
item_cols = [col for col in data.columns if col.startswith('q_')]

# Context 기반 정보 출력
print(f"=== Context에서 로드된 정보 ===")
print(f"응답자 수: {ctx['data']['n_respondents']:,}")
print(f"평균 응답: {ctx['data']['avg_responses']}개")
print(f"결측률: {ctx['data']['missing_rate']:.1%}")

# QC 기준 설정 (Context 기반)
MIN_RESPONSES = max(10, int(ctx['data']['avg_responses'] * 0.1))
print(f"\nQC 기준 (최소 응답): {MIN_RESPONSES}개")

=== Context에서 로드된 정보 ===
응답자 수: 23,679
평균 응답: 86개
결측률: 87.6%

QC 기준 (최소 응답): 10개


---
## Part 2: QC (Quality Control)

### 2.1 응답 수 부족 탐지

In [4]:
# 응답자별 응답 수 계산
responses_per_person = data[item_cols].notna().sum(axis=1)

# 응답 부족 (최소 기준 미달)
low_response_mask = responses_per_person < MIN_RESPONSES
low_response_ids = data.loc[low_response_mask, 'RID'].tolist()

print(f"응답 부족 ({MIN_RESPONSES}개 미만): {len(low_response_ids)}명")
print(f"비율: {len(low_response_ids)/len(data):.2%}")

응답 부족 (10개 미만): 1명
비율: 0.00%


### 2.2 Straight-lining 탐지

In [5]:
# Straight-lining: 응답한 문항 중 표준편차가 0인 경우
def check_straight_lining(row):
    responses = row.dropna()
    if len(responses) < 10:  # 응답이 10개 미만이면 판단 불가
        return False
    return responses.std() == 0

straight_line_mask = data[item_cols].apply(check_straight_lining, axis=1)
straight_line_ids = data.loc[straight_line_mask, 'RID'].tolist()

print(f"Straight-lining 의심: {len(straight_line_ids)}명")

Straight-lining 의심: 31명


### 2.3 QC 결과 요약

In [6]:
# 제외 대상 통합 (중복 제거)
exclude_ids = list(set(low_response_ids + straight_line_ids))

print("=" * 50)
print("QC 결과 요약")
print("=" * 50)
print(f"전체 응답자: {len(data):,}명")
print(f"응답 부족: {len(low_response_ids)}명")
print(f"Straight-lining: {len(straight_line_ids)}명")
print(f"제외 대상 (합계): {len(exclude_ids)}명")
print(f"유효 응답자: {len(data) - len(exclude_ids):,}명")
print("=" * 50)

# QC 통과한 데이터만 사용
data_clean = data[~data['RID'].isin(exclude_ids)].copy()
print(f"\n✅ QC 통과 데이터: {len(data_clean):,}명")

QC 결과 요약
전체 응답자: 23,679명
응답 부족: 1명
Straight-lining: 31명
제외 대상 (합계): 32명
유효 응답자: 23,647명

✅ QC 통과 데이터: 23,647명


---
## Part 3: 점수 계산 함수 정의

> **참조**: `reports/preprocessing_guide.md` 섹션 2, 3

In [7]:
def calculate_scale_score(df, keys, scale_name):
    """
    채점 키를 사용해 척도 점수 계산
    (preprocessing_guide.md 기반)
    
    Parameters:
    - df: 응답 데이터
    - keys: 채점 키 (superKey696.csv)
    - scale_name: 척도명 (예: 'NEO_E')
    
    Returns:
    - 척도 점수 Series
    
    채점 규칙:
    - 1: 정채점 (원점수 그대로)
    - -1: 역채점 (7 - 원점수)
    - 0: 해당 없음
    """
    # 해당 척도에 속하는 문항 찾기
    scale_items = keys.index[keys[scale_name] != 0].tolist()
    weights = keys.loc[scale_items, scale_name]
    
    # 데이터에 있는 문항만 필터링
    available_items = [q for q in scale_items if q in df.columns]
    
    if not available_items:
        return pd.Series([np.nan] * len(df), index=df.index)
    
    # 역채점 적용 (6점 척도: 7 - 원점수)
    subset = df[available_items].copy()
    for item in available_items:
        if weights[item] == -1:
            subset[item] = 7 - subset[item]
    
    # 평균 계산 (결측 무시)
    return subset.mean(axis=1, skipna=True)


def z_score(series):
    """
    표준화 (z-score)
    Ideology, Honesty-Humility 계산에 사용
    """
    return (series - series.mean()) / series.std()


print("✅ 점수 계산 함수 정의 완료")

✅ 점수 계산 함수 정의 완료


---
## Part 4: Big Five 점수 계산

| 척도 | 컬럼명 | 설명 |
|------|--------|------|
| Openness | NEO_O | 개방성 |
| Conscientiousness | NEO_C | 성실성 |
| Extraversion | NEO_E | 외향성 |
| Agreeableness | NEO_A | 우호성 |
| Neuroticism | NEO_N | 신경증 |

In [10]:
# 점수 DataFrame 생성
scores = pd.DataFrame()
scores['RID'] = data_clean['RID']

# Big Five 점수 계산
print("Big Five 점수 계산 중...")
scores['NEO_O'] = calculate_scale_score(data_clean, keys, 'NEO_O').values
scores['NEO_C'] = calculate_scale_score(data_clean, keys, 'NEO_C').values
scores['NEO_E'] = calculate_scale_score(data_clean, keys, 'NEO_E').values
scores['NEO_A'] = calculate_scale_score(data_clean, keys, 'NEO_A').values
scores['NEO_N'] = calculate_scale_score(data_clean, keys, 'NEO_N').values

print("\n=== Big Five 점수 통계 ===")
print(scores[['NEO_O', 'NEO_C', 'NEO_E', 'NEO_A', 'NEO_N']].describe().round(2))

Big Five 점수 계산 중...

=== Big Five 점수 통계 ===
          NEO_O     NEO_C     NEO_E     NEO_A     NEO_N
count  23399.00  23417.00  23357.00  23437.00  23371.00
mean       4.31      4.23      3.87      4.23      3.35
std        0.83      0.86      0.92      0.82      0.98
min        1.00      1.00      1.00      1.00      1.00
25%        3.78      3.67      3.25      3.71      2.67
50%        4.33      4.25      3.89      4.25      3.33
75%        4.89      4.83      4.50      4.80      4.00
max        6.00      6.00      6.00      6.00      6.00


In [9]:
scores.head()  # 처음 5명

,RID,NEO_O,NEO_C,NEO_E,NEO_A,NEO_N
0,111610,5.000000,4.000000,4.625000,4.000000,2.000000
1,236351,3.750000,3.750000,4.600000,3.500000,4.750000
2,258633,4.611111,3.588235,4.470588,3.750000,4.111111
3,273126,5.000000,4.000000,3.666667,3.333333,4.222222
4,371933,4.800000,3.500000,3.285714,4.500000,4.666667


---
## Part 5: Ideology 점수 계산

> **공식** (preprocessing_guide.md):  
> `Ideology = mean( z(MPQtr), z(NEOo6) * -1 )`
>
> - MPQtr: MPQ Traditionalism (보수성)
> - NEOo6: NEO Liberalism (자유주의) → 역채점하여 보수성 지표로 변환

In [11]:
# Ideology 구성 요소 계산
print("Ideology 점수 계산 중...")
scores['MPQtr'] = calculate_scale_score(data_clean, keys, 'MPQtr').values
scores['NEOo6'] = calculate_scale_score(data_clean, keys, 'NEOo6').values

# Ideology = mean(z(MPQtr), z(NEOo6) * -1)
scores['Ideology'] = (z_score(scores['MPQtr']) + z_score(scores['NEOo6']) * -1) / 2

print(f"\nIdeology 점수 계산 완료")
print(f"  - 유효 N: {scores['Ideology'].notna().sum():,}")
print(f"  - 평균: {scores['Ideology'].mean():.3f}")
print(f"  - 표준편차: {scores['Ideology'].std():.3f}")

Ideology 점수 계산 중...

Ideology 점수 계산 완료
  - 유효 N: 11,825
  - 평균: 0.033
  - 표준편차: 0.969


---
## Part 6: Honesty-Humility 점수 계산

> **공식** (preprocessing_guide.md):  
> `Honesty-Humility = mean( z(NEOa2), z(NEOa4), z(HEXACO_H) )`
>
> - NEOa2: NEO Morality (도덕성)
> - NEOa4: NEO Modesty (겸손)
> - HEXACO_H: HEXACO Honesty-Humility

In [12]:
# Honesty-Humility 구성 요소 계산
print("Honesty-Humility 점수 계산 중...")
scores['NEOa2'] = calculate_scale_score(data_clean, keys, 'NEOa2').values
scores['NEOa4'] = calculate_scale_score(data_clean, keys, 'NEOa4').values
scores['HEXACO_H'] = calculate_scale_score(data_clean, keys, 'HEXACO_H').values

# Honesty-Humility = mean(z(NEOa2), z(NEOa4), z(HEXACO_H))
scores['Honesty_Humility'] = (
    z_score(scores['NEOa2']) + 
    z_score(scores['NEOa4']) + 
    z_score(scores['HEXACO_H'])
) / 3

print(f"\nHonesty-Humility 점수 계산 완료")
print(f"  - 유효 N: {scores['Honesty_Humility'].notna().sum():,}")
print(f"  - 평균: {scores['Honesty_Humility'].mean():.3f}")
print(f"  - 표준편차: {scores['Honesty_Humility'].std():.3f}")

Honesty-Humility 점수 계산 중...

Honesty-Humility 점수 계산 완료
  - 유효 N: 12,828
  - 평균: -0.003
  - 표준편차: 0.727


---
## Part 7: 결과 저장

In [13]:
# 최종 점수 컬럼 선택 (7개 broadband 측정치)
final_cols = ['RID', 'NEO_O', 'NEO_C', 'NEO_E', 'NEO_A', 'NEO_N', 'Ideology', 'Honesty_Humility']
scores_final = scores[final_cols].copy()

# processed 폴더 생성
os.makedirs('data/processed', exist_ok=True)

# 저장
scores_final.to_csv('data/processed/sapa_scores.csv', index=False)

print("=" * 50)
print("✅ 점수 저장 완료!")
print("=" * 50)
print(f"파일: data/processed/sapa_scores.csv")
print(f"응답자 수: {len(scores_final):,}")
print(f"척도 수: {len(final_cols) - 1}개")
print(f"\n계산된 척도:")
for col in final_cols[1:]:
    valid_n = scores_final[col].notna().sum()
    print(f"  - {col}: {valid_n:,}명 유효")

✅ 점수 저장 완료!
파일: data/processed/sapa_scores.csv
응답자 수: 23,647
척도 수: 7개

계산된 척도:
  - NEO_O: 23,399명 유효
  - NEO_C: 23,417명 유효
  - NEO_E: 23,357명 유효
  - NEO_A: 23,437명 유효
  - NEO_N: 23,371명 유효
  - Ideology: 11,825명 유효
  - Honesty_Humility: 12,828명 유효
